In [ ]:
#default_exp speclib.library_base

In [ ]:
#export 
import pandas as pd
import numpy as np
import typing

import alphabase.peptide.fragment as fragment
from alphabase.io.hdf import HDF_File

class SpecLibBase(object):
    def __init__(self,
        # e.g. ['b_1','b_2','y_1','y_2', ...]
        charged_frag_types:typing.List[str], 
        min_frag_mz = 100, max_frag_mz = 2000,
        min_precursor_mz = 400, max_precursor_mz = 6000,
    ):
        self.charged_frag_types = charged_frag_types
        self._precursor_df:pd.DataFrame = None
        self._fragment_inten_df:pd.DataFrame = None
        self._fragment_mass_df:pd.DataFrame = None
        self.min_frag_mz = min_frag_mz
        self.max_frag_mz = max_frag_mz
        self.min_precursor_mz = min_precursor_mz
        self.max_precursor_mz = max_precursor_mz

    @property
    def precursor_df(self):
        return self._precursor_df

    @property
    def fragment_mass_df(self):
        return self._fragment_mass_df

    @property
    def fragment_inten_df(self):
        return self._fragment_inten_df

    def clip_precursor_by_mz_(self):
        ''' 
        Clip self._precursor_df inplace
        '''
        self._precursor_df = self._precursor_df[
            (self._precursor_df['precursor_mz']>=self.min_precursor_mz)&
            (self._precursor_df['precursor_mz']<=self.max_precursor_mz)
        ]
        self._precursor_df.reset_index(drop=True, inplace=True)

    def clip_inten_by_fragment_mass_(self):
        ''' 
        Clip self._fragment_inten_df inplace. 
        All clipped intensities are set as zeros.
        A more generic way is to use a mask.
        '''
        self._fragment_inten_df[
            (self._fragment_mass_df<self.min_frag_mz)|
            (self._fragment_mass_df>self.max_frag_mz)
        ] = 0

    def clip_inten_by_fragment_mass(self)->pd.DataFrame:
        df = self._fragment_inten_df.copy()
        df[
            (self._fragment_mass_df<self.min_frag_mz)|
            (self._fragment_mass_df>self.max_frag_mz)
        ] = 0
        return df
    
    def load_precursor_df(self, 
        precursor_files, **kwargs
    ):
        self._load_precursor_df(precursor_files, **kwargs)
        self.clip_precursor_by_mz_()

    def _load_precursor_df(self, precursor_files, **kwargs):
        '''
        All sub-class must reimplement this method
        '''
        raise NotImplementedError(
            f'Sub-class of "{self.__class__}" must re-implement "_load_precursor_df()"'
        )

    def load_fragment_df(self, **kwargs):
        self.load_fragment_mass_df(**kwargs)
        self.load_fragment_inten_df(**kwargs)

    def flatten_fragment_data(
        self
    )->typing.Tuple[np.array, np.array]:
        '''
        Create flattened (1-D) np.array for mass and intensity 
        dataframes, respectively. The arrays are references to 
        original data, that means: 
          1. This method is fast; 
          2. Changing the array values will change the df values. 
        They can be unraveled back using:
          `array.reshape(len(self._fragment_mass_df.columns), -1)`

        Retruns:
            np.array: 1-D flattened mass array (a reference to 
            original mass df data)
            np.array: 1-D flattened intensity array (a reference to 
            original intensity df data)
        '''
        return (
            self._fragment_mass_df.values.reshape(-1),
            self._fragment_inten_df.values.reshape(-1)
        )

    def load_fragment_inten_df(self, **kwargs):
        '''
        All sub-class must re-implement this method. 
        Fragment intensities can be predicted or from AlphaPept, or ...
        '''
        raise NotImplementedError(
            f'Sub-class of "{self.__class__}" must re-implement "load_fragment_inten_df()"'
        )

    def load_fragment_mass_df(self):
        need_clip = (
            False if 
            'precursor_mz' in self._precursor_df.columns 
            else True
        )

        (
            self._precursor_df, self._fragment_mass_df
        ) = fragment.get_fragment_mass_dataframe(
            self._precursor_df, self.charged_frag_types
        )
        if need_clip: self.clip_precursor_by_mz_()

    def save_hdf(self, hdf_file):
        _hdf = HDF_File(
            hdf_file, 
            read_only=False, 
            truncate=True,
            delete_existing=True
        )
        _hdf.precursor_df = self._precursor_df
        _hdf.fragment_mass_df = self._fragment_mass_df
        _hdf.fragment_inten_df = self._fragment_inten_df

    def load_hdf(self, hdf_file):
        self._hdf = HDF_File(
            hdf_file,
        )
        self._precursor_df = self._hdf.precursor_df.values
        self._fragment_mass_df = self._hdf.fragment_mass_df.values
        self._fragment_inten_df = self._hdf.fragment_inten_df.values